In [1]:
import pandas as pd
import numpy as np
import ast

Data Cleaning

In [2]:
labels = pd.read_csv('data/small_data/labels.csv', index_col=0)
print(labels.shape)
labels.head()

(2218, 7)


,index,genes,sex,origin,price,birth,url
0,0-0,"['Yellow Belly', 'Pastel', 'Het Puzzle', '50% ...",female,Self Produced,650.0,7th April 2023,https://www.morphmarket.com/us/c/reptiles/pyth...
1,1-0,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...
2,1-1,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...
3,2-0,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...
4,2-1,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...


In [3]:
clean_genes = []
list_genes = [ast.literal_eval(gene) for gene in labels['genes']]
for lst in list_genes:
    for element in lst:
        if "Het" in element:
            clean_genes.append('Het' + ' '.join(element.split('Het')[1:]))
        else:
            clean_genes.append(element)

clean_possible_genes = list(set(clean_genes))
print(f'Number of possible genes: {len(clean_possible_genes)}')
clean_possible_genes[:5]

Number of possible genes: 146


['Satin', 'Het Clown', 'Het Toffee', 'Het Piebald', 'Specter']

In [4]:
gene_extension_df = pd.DataFrame(np.zeros([labels.shape[0], len(clean_possible_genes)]), dtype=int, columns=clean_possible_genes)
labels_extended = pd.concat([labels, gene_extension_df], axis=1)
labels_extended.head()

,index,genes,sex,origin,price,birth,url,Satin,Het Clown,Het Toffee,...,Butter,Blade,Albino,Super Orange Dream,Vanilla,Red Gene,Pastel,Asphalt,Super Gravel,Super Spotnose
0,0-0,"['Yellow Belly', 'Pastel', 'Het Puzzle', '50% ...",female,Self Produced,650.0,7th April 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1-0,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1-1,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2-0,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2-1,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
start_row = 0
count = 0
for gene_col in clean_genes:
    labels_extended.loc[start_row, gene_col] = 1
    count += 1
    if count == [len(lst) for lst in list_genes][start_row]:
        start_row += 1
        count = 0
labels_extended.head()

,index,genes,sex,origin,price,birth,url,Satin,Het Clown,Het Toffee,...,Butter,Blade,Albino,Super Orange Dream,Vanilla,Red Gene,Pastel,Asphalt,Super Gravel,Super Spotnose
0,0-0,"['Yellow Belly', 'Pastel', 'Het Puzzle', '50% ...",female,Self Produced,650.0,7th April 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1-0,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1-1,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2-0,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,2-1,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...,0,1,0,...,1,0,0,0,0,0,0,0,0,0
